In [1]:
# All system imports
import logging
import os.path
import time
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['image.cmap'] = 'gray'


In [2]:
# Define our identity based on our hostname, connect to the right PCM.

hostname = !hostname
hostname = os.path.splitext(hostname[0])[0]
spectroId = int(hostname[-1])
band = hostname[-2]
cam = '%s%d' % (band, spectroId)

print "hostname=%s band=%s spectroId=%s" % (hostname, band, spectroId)

import xcuActor.Controllers.PCM as PCM

pcm = PCM.PCM(host='pcm-%s' % (cam))
pcm.logger.setLevel(logging.INFO)

In [3]:
import fpga.ccdFuncs as ccdFuncs
import fpga.nbFuncs as nbFuncs
import testing.ccdProcedures as ccdTests

In [4]:
# Connect to the FPGA

import fpga.ccd as fpgaMod
reload(fpgaMod)

ccd = fpgaMod.CCD(spectroId=spectroId, dewarId=band, adc18bit=1)
ccd.pciReset()

print "FPGA version: 0x%08x" % ccd.peekWord(0)

In [5]:
# connect to the FEE

import fee.feeControl as feeMod
reload(feeMod)

pcm.powerOn('fee')
time.sleep(3.5)

logLevel = 20
fee = feeMod.FeeControl(logLevel=logLevel)

In [6]:
# Fetch serial numbers and temperatures
for thing in 'FEE', 'ADC', 'PA0', 'CCD0', 'CCD1':
    print "serial %-5s: %s" % (thing, fee.doGet('serial',  thing))
        
print
for probe in 'FEE', 'PA', 'ccd0', 'ccd1':
    print "temp   %-5s: %s" % (probe, fee.sendCommandStr('rt,%s' % (probe)))


In [7]:
feeMod.fee.statusAsCards()

In [8]:
chans = 'ch0', 'ch1'
modes = 'erase', 'expose', 'read', 'wipe', 'idle'

for mode in modes:
    fee.setMode(mode)
    time.sleep(1)
    
    for chan in chans:
        v = fee.sendCommandStr('rb,all,%s' % (chan))
        readAll = np.array([float(b) for b in v.split(',')])
        v = fee.sendCommandStr('gb,all,%s' % (chan))
        getAll = np.array([float(b) for b in v.split(',')])
        
        offby = np.fabs(getAll-readAll)/getAll
                        
        print("%07s %s get : %s" % (mode, chan, ' '.join(["%6.2f" % (f) for f in getAll])))
        print("%07s %s read: %s" % (mode, chan, ' '.join(["%6.2f" % (f) for f in readAll])))
        print("%16s: %s" % (('BAD' if np.any(offby > 0.02) else 'OK'), 
                            ' '.join(["%6.2f" % (f) for f in offby])))
        print


In [9]:
reload(ccdFuncs)
reload(nbFuncs)

doSave = True
benchTest = True
if benchTest:
    nrows = 400               # How many rows to read out.
    ncols = 400
else:
    nrows = None
    ncols = None
    
argDict = dict(everyNRows=100)

comment = ''

doFile = False # '/data/pfs/2016-02-26/PFJA00737691.fits'
print "doFile=%s" % (doFile,)
if doFile:
    im = fitsio.read(doFile)
    files = [doFile]
else:
    im, files = ccdFuncs.fullExposure('bias', ccd, expTime=0.0,
                                      nrows=nrows, ncols=ncols, 
                                      doSave=doSave,
                                      comment=comment)
nrows, ncols = im.shape
ncols /= 8
        
print files[0]

figWidth = 12

selectRows = slice(0,None)
selectCols = slice(0,None) # None # slice(20,ncols)
pcols = np.arange(50,ncols-2)    # Which cols to plot
prow = nrows/2
prows = [2,prow,nrows-1]       # which rows to plot
statCols = np.arange(20,ncols-20) # Which cols to use for stats

if selectRows is not None:
    im = im[selectRows]
if selectCols is not None:
    im = im[:,selectCols]
    
# im = im.astype('i4')

print "statCols: %d %d" % (statCols[0], statCols[-1])
means,devs = nbFuncs.ampStats(im, ccd=ccd, cols=statCols)
print "means:", nbFuncs.fmtArr(means, format="%0.3f")
print "devs: ", nbFuncs.fmtArr(devs, format="%0.3f")
    
if len(files) > 0:
    path = files[0]
    imName = '/'.join(path.split('/')[-2:])
    print "file: %s" % (path)

# plot one or more rows across the entire image
nbFuncs.plotRows(im, prows, imName=imName, figWidth=figWidth, pixRange=200 if siggenAmp is not None else None)

# plot one row for each the amps
nbFuncs.plotAmps(im, row=prow, cols=pcols, 
                 plotOffset=10, fig=None, figWidth=figWidth)

# show a grid of histograms.
hists = nbFuncs.ampHistGrid(im, ccd, cols=statCols, histRange=20, figWidth=figWidth)

# show a grid of raw amp images.
nbFuncs.rawAmpGrid(im, ccd, cols=slice(50,None), figWidth=figWidth, showFfts=True)

#fig.colorbar(pp)
#plt.tight_layout()